In [1]:
import numpy as np
import pandas as pd 

In [4]:
def read_data(file_path, label):
    num_roads = 24  
#     saved_path = '/Users/gengyunxin/Documents/项目/traffic_model/test/data/processed_finetune_seattle_72.csv'
    df = pd.read_csv(file_path, names=None).values
  
    df_T = df.transpose() # (24, 12384)
    # 每条样本的长度要综合考虑，不能存在两条道路的数据混合在一条样本中的情况
    # 即能被12384整除；
    # 还要和小时、天等实际意义挂钩；
    # 同时要考虑一条样本（sentence）中“词”的数量要合理
    
    # 每条路的数据独立做归一化
    raw = []
    for i in range(num_roads):
        array = df_T[i]
        max = array.max()
        min = array.min()
        normalized = ((array - min) / (max - min)).tolist()
        raw.append(normalized)

    df_norm = np.array(raw)
    
    data_quarter = df_norm.reshape(-1, 72) # 每条样本72个点,即1/4天 

    num_rows = data_quarter.shape[0]
    if label == 1:
        labels = np.ones((1, num_rows))
    else:
        labels = np.zeros((1, num_rows))
    data_with_label = np.insert(data_quarter, 72, values=labels, axis=1) 
    
    return data_with_label

In [5]:
workday_file = '/Users/gengyunxin/Documents/项目/traffic_model/data/California/combine_E_workday_n0.csv'
weekend_file = '/Users/gengyunxin/Documents/项目/traffic_model/data/California/combine_E_weekend_n0.csv'
# weekday->1 ; weekend->0
workday_with_label = read_data(workday_file, label=1) # (4128, 73)
weekend_with_label = read_data(weekend_file, label=0) # (1728, 73)

In [6]:
workday_with_label.shape

(4128, 73)

In [7]:
weekend_with_label.shape

(1728, 73)

In [9]:
all_with_label = np.append(workday_with_label, weekend_with_label, axis=0) # (5856, 73)

(5856, 73)

In [10]:
df = pd.DataFrame(all_with_label)
df.to_csv('/Users/gengyunxin/Documents/项目/traffic_model/test/data/processed_finetune_seattle_72.csv', index=0, header=0)